In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm
import csv
from torchsummary import summary
import torchvision

data_dir = 'GTSRB/Training'


In [2]:
'''
All about the classification model and training
'''

# Define the transformations.

data_transforms = transforms.Compose([
    transforms.Resize([32, 32]),
    transforms.RandomPerspective(distortion_scale=0.6, p=0.4),
    transforms.ToTensor()
    ])


# Define path of training data

train_data_path = data_dir
train_data = datasets.ImageFolder(root = train_data_path, transform = data_transforms)

# Divide data into training and validation (0.8 and 0.2)
ratio = 0.8
n_train_examples = int(len(train_data) * ratio)
n_val_examples = len(train_data) - n_train_examples

train_data, val_data = data.random_split(train_data, [n_train_examples, n_val_examples])

# Defining the CNN model

class CNN1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5,1)
        nn.init.kaiming_normal_(self.conv1.weight) # Weight initialization
        self.conv2 = nn.Conv2d(32,32,3,1)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.conv3 = nn.Conv2d(32,64,1,1)
        nn.init.kaiming_normal_(self.conv3.weight)
        self.conv4 = nn.Conv2d(64,64,3,1)
        nn.init.kaiming_normal_(self.conv4.weight)
        
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4_bn = nn.BatchNorm2d(64)
        
        self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2_bn = nn.BatchNorm1d(43)

        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 43)
        
        self.dropout=nn.Dropout(0.2)

    def forward(self, x):
        
        x = F.relu(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x))))))
        x = self.pool(x)
        
        x = self.dropout(x)
        
        x = F.relu(self.conv4_bn(self.conv4(F.relu(self.conv3_bn(self.conv3(x))))))
        x = self.pool(x)
        x = self.dropout(x)
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
    
        return x
    
def check_accuracy(model, dataloader, model_forward='loss'):
    y_pred = []
    y = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if model_forward!='loss':
        model.eval()
    for batch, (inputs, labels) in tqdm(enumerate(dataloader, 0)):
        inputs = inputs.to(device)
        if model_forward=='loss':
            scores = model.loss(inputs)
        elif model_forward=='forward':
            with torch.no_grad():
                scores = model(torch.tensor(inputs)).detach()
        scores=scores.cpu()
        scores=scores.numpy()
        y.append(labels)
        y_pred.append(np.argmax(scores, axis=1))
        
    y_pred = np.hstack(y_pred)
    y = np.hstack(y)
    acc = np.mean(y_pred == y)

    return acc*100

# Returns the training model
def training(epochs, batch_size, results, lr=0.001):
    classes=43
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader = data.DataLoader(train_data, shuffle=True, batch_size = batch_size)
    val_loader = data.DataLoader(val_data, shuffle=True, batch_size = batch_size)

    CNNtest1 = CNN1()
    CNNtest1.to(device)
    criterion = nn.CrossEntropyLoss() # Criterion for the model
    optimizer = optim.Adam(CNNtest1.parameters(), lr) # Initialize the optimizer
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)# Initialize the scheduler
    
    key = lr
    results[key] = np.zeros(epochs)
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):            
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = CNNtest1(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        running_loss = 0.0
        scheduler.step()
        test_acc = check_accuracy(CNNtest1, val_loader, 'forward')
        
        results[key][epoch] = test_acc
    return CNNtest1, results




In [5]:
# Training the model
results = {}
epochs = 30
CNNmodel, results = training(epochs, 128, results, 0.001)

In [7]:
# Plotting model accuarcy
ep = [i for i in range(1,epochs+1)]
for key, val in results.items():
    plt.plot(ep, val, label=f"Learning Rate {key}: {val[-1]:.2f}")
plt.legend(fontsize=10, title_fontsize=15)
plt.title("full data (accuracy)")
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [9]:
# Save the model
torch.save(CNNmodel, r'./CNN2waug.pkl')
